In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import decomposition
from collections import Counter
from sklearn.model_selection import train_test_split

In [ ]:
# QuadTree implementation
class QuadTree:
    def __init__(self, x_lo, y_lo, x_hi, y_hi, points, max_points=20, depth=0, max_depth=20, parent=None):
        self.x_lo = x_lo
        self.y_lo = y_lo
        self.x_hi = x_hi
        self.y_hi = y_hi
        self.parent = parent
        self.children = []   
        self.points = points
        self.max_points = max_points
        self.depth = depth
        self.max_depth = max_depth

        # Decide whether to split
        if len(self.points) > self.max_points and self.depth < self.max_depth:
            self._split()

    def _split(self):
        # Compute midpoints
        x_mid = 0.5 * (self.x_lo + self.x_hi)
        y_mid = 0.5 * (self.y_lo + self.y_hi)

        # Initialize children with empty point lists
        nw_points = []
        ne_points = []
        sw_points = []
        se_points = []

        # Assign points to quadrants
        for (x, y, label, idx) in self.points:
            if x <= x_mid and y >= y_mid:
                nw_points.append((x, y, label, idx))
            elif x > x_mid and y >= y_mid:
                ne_points.append((x, y, label, idx))
            elif x <= x_mid and y < y_mid:
                sw_points.append((x, y, label, idx))
            else:
                se_points.append((x, y, label, idx))

        # Create child nodes
        self.children = [
            QuadTree(self.x_lo, y_mid,   x_mid,   self.y_hi, nw_points, self.max_points, self.depth+1, self.max_depth, parent=self),
            QuadTree(x_mid,   y_mid,   self.x_hi, self.y_hi, ne_points, self.max_points, self.depth+1, self.max_depth, parent=self),
            QuadTree(self.x_lo, self.y_lo, x_mid, y_mid,     sw_points, self.max_points, self.depth+1, self.max_depth, parent=self),
            QuadTree(x_mid,   self.y_lo, self.x_hi, y_mid,   se_points, self.max_points, self.depth+1, self.max_depth, parent=self),
        ]

        self.points = []

    # Helper methods
    def is_leaf(self):
        return len(self.children) == 0

    def contains(self, x, y):
        """Check if this bounding box contains point (x, y)."""
        return (self.x_lo <= x <= self.x_hi) and (self.y_lo <= y <= self.y_hi)

    def small_containing_quadtree(self, x, y):
        """
        Return the smallest subtree (leaf) whose box contains (x, y).
        """
        if not self.contains(x, y):
            return None
        if self.is_leaf():
            return self
        for child in self.children:
            if child.contains(x, y):
                return child.small_containing_quadtree(x, y)
        return self  

    def _within_distance(self, x, y, d):
        """
        Return True if the rectangle of this node is within distance d of point (x, y),
        meaning the circle of radius d around (x, y) intersects the rectangle.
        """
        # Find closest point in rectangle to (x, y)
        closest_x = min(max(x, self.x_lo), self.x_hi)
        closest_y = min(max(y, self.y_lo), self.y_hi)
        dx = closest_x - x
        dy = closest_y - y
        return (dx*dx + dy*dy) <= d*d

    def leaves_within_distance(self, x, y, d, result=None):
        """
        Collect all leaf nodes whose bounding box is within distance d of (x, y).
        """
        if result is None:
            result = []

        if not self._within_distance(x, y, d):
            return result

        if self.is_leaf():
            result.append(self)
        else:
            for child in self.children:
                child.leaves_within_distance(x, y, d, result)

        return result

    def all_points(self):
        """
        Return all points in this subtree as a list.
        """
        if self.is_leaf():
            return list(self.points)
        else:
            pts = []
            for child in self.children:
                pts.extend(child.all_points())
            return pts


# k-NN using QuadTree for neighbor search
class KNNQuadTreeClassifier:
    def __init__(self, k):
        self.k = k
        self.quadtree = None
        self.labels_array = None
        self.X_train = None

    def fit(self, X, y):
        """
        X: array of shape (n_samples, 2)  (pc0, pc1)
        y: array of shape (n_samples,)    (labels)
        """
        self.X_train = X
        self.labels_array = np.array(y)

        x_coords = X[:, 0]
        y_coords = X[:, 1]

        # Root bounding box with small margins
        margin = 1e-6
        x_lo, x_hi = x_coords.min() - margin, x_coords.max() + margin
        y_lo, y_hi = y_coords.min() - margin, y_coords.max() + margin

        # points: (x, y, label, index)
        points = [(x_coords[i], y_coords[i], self.labels_array[i], i) for i in range(len(self.labels_array))]

        self.quadtree = QuadTree(x_lo, y_lo, x_hi, y_hi, points, max_points=20, max_depth=20)

    def _find_k_nearest_indices(self, x, y):
        """
        Use expanding-radius search over the quadtree to collect at least k candidate points,
        then pick the k closest among them.
        """
        # Start with a small radius based on data spread
        if self.X_train is None:
            raise ValueError("Model not fitted yet.")

        # Rough scale for initial radius
        spread = np.std(self.X_train, axis=0).mean()
        if spread == 0:
            spread = 1.0
        radius = spread * 0.5

        candidates = []
        max_iterations = 20
        factor = 2.0

        for _ in range(max_iterations):
            leaves = self.quadtree.leaves_within_distance(x, y, radius, result=[])
            candidates = []
            for leaf in leaves:
                candidates.extend(leaf.points)
            if len(candidates) >= self.k:
                break
            radius *= factor

        if len(candidates) == 0:
            # fallback: use all training points
            candidates = [(self.X_train[i, 0], self.X_train[i, 1], self.labels_array[i], i)
                          for i in range(len(self.labels_array))]

        # Compute distances to all candidates
        cand_coords = np.array([[px, py] for (px, py, lab, idx) in candidates])
        cand_indices = np.array([idx for (px, py, lab, idx) in candidates])
        diffs = cand_coords - np.array([x, y])
        dists = np.sum(diffs**2, axis=1)

        if len(dists) <= self.k:
            # fewer than or equal to k candidates; just return all
            return cand_indices

        # find indices of the smallest k distances
        min_i = np.argpartition(dists, self.k)[:self.k]
        return cand_indices[min_i]

    def predict_one(self, x, y):
        knn_indices = self._find_k_nearest_indices(x, y)
        knn_labels = self.labels_array[knn_indices]
        most_common = Counter(knn_labels).most_common(1)[0][0]
        return most_common

    def predict(self, X):
        preds = []
        for i in range(X.shape[0]):
            preds.append(self.predict_one(X[i, 0], X[i, 1]))
        return np.array(preds)


# simple k-NN for testing 
def knn_predict(X_train, y_train, X_test, k):
    predictions = []
    for i in range(X_test.shape[0]):
        diff = X_train - X_test[i]
        dists = np.sum(diff**2, axis=1)
        if len(dists) <= k:
            k_idx = np.arange(len(dists))
        else:
            k_idx = np.argpartition(dists, k)[:k]
        k_labels = y_train[k_idx]
        most_common = Counter(k_labels).most_common(1)[0][0]
        predictions.append(most_common)
    return np.array(predictions)


# Main analysis pipeline
def main():
    # 1. Load data
    data = pd.read_excel('Rice_Cammeo_Osmancik.xlsx')
    feature_cols = [col for col in data.columns if col != "Class"]
    class_col = "Class"

    X = data[feature_cols].values
    y = data[class_col].values

    # 2. Train-test split
    X_train_raw, X_test_raw, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )

    # 3. Normalize based on training set mean and std
    mean = X_train_raw.mean(axis=0)
    std = X_train_raw.std(axis=0, ddof=0)
    std[std == 0] = 1.0  # avoid division by zero

    X_train_norm = (X_train_raw - mean) / std
    X_test_norm = (X_test_raw - mean) / std

    # 4. PCA to 2D
    pca = decomposition.PCA(n_components=2)
    X_train_reduced = pca.fit_transform(X_train_norm)
    X_test_reduced = pca.transform(X_test_norm)

    # Also reduce full dataset for plotting
    X_all_norm = (X - mean) / std
    X_all_reduced = pca.transform(X_all_norm)

    # 5. Scatterplot
    pc0 = X_all_reduced[:, 0]
    pc1 = X_all_reduced[:, 1]

    plt.figure(figsize=(6, 6))
    classes = np.unique(y)
    for c in classes:
        mask = (y == c)
        plt.scatter(pc0[mask], pc1[mask], alpha=0.5, label=str(c))
    plt.xlabel('PC 0')
    plt.ylabel('PC 1')
    plt.title('Rice PCA (2D) colored by class')
    plt.legend()
    plt.tight_layout()
    plt.savefig('rice_pca_scatter.png', dpi=150)
    plt.close()

    # 6. Fit k-NN with QuadTree (k=1 and k=5)
    for k_val in [1, 5]:
        print(f"QuadTree k-NN with k={k_val}")
        knn_qt = KNNQuadTreeClassifier(k=k_val)
        knn_qt.fit(X_train_reduced, y_train)
        y_pred_qt = knn_qt.predict(X_test_reduced)

        # Confusion matrix
        cm = pd.crosstab(pd.Series(y_test, name='True'),
                         pd.Series(y_pred_qt, name='Predicted'))

        print("Confusion matrix:")
        print(cm)
    
        # compare with simple k-NN
        y_pred_brute = knn_predict(X_train_reduced, y_train, X_test_reduced, k_val)
        agreement = np.mean(y_pred_brute == y_pred_qt)
        print(f"Agreement QuadTree vs Simple k-NN for k={k_val}: {agreement:.4f}")


if __name__ == "__main__":
    main()

QuadTree k-NN with k=1
Confusion matrix:
Predicted  Cammeo  Osmancik
True                       
Cammeo        412        77
Osmancik       58       596
Agreement QuadTree vs Simple k-NN for k=1: 0.9991
QuadTree k-NN with k=5
Confusion matrix:
Predicted  Cammeo  Osmancik
True                       
Cammeo        426        63
Osmancik       41       613
Agreement QuadTree vs Simple k-NN for k=5: 1.0000
